<a href="https://colab.research.google.com/github/ReuvenMuller/IT7143/blob/main/IT7143_Assignment2_rmuller7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 2**

## 1. Import the data

Looking at the raw data file:

*   The data contains headers
*   The data is is comma seperated

We can read the data as is using pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Documents/Education/KSU/Summer/IT7143 - Cloud Analytics Technology/Week 1/garments_worker_productivity.csv')
data

Mounted at /content/drive


,quarter,department,day,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,Quarter1,sweing,Thursday,8.0,0.80,26.16,1108.0,7080.0,98,0.0,0.0,0.0,59.0,0.940725
1,Quarter1,finishing,Thursday,1.0,0.75,3.94,NaN,960.0,0,0.0,0.0,0.0,8.0,0.886500
2,Quarter1,sweing,Thursday,11.0,0.80,11.41,968.0,3660.0,50,0.0,0.0,0.0,30.5,0.800570
3,Quarter1,sweing,Thursday,12.0,0.80,11.41,968.0,3660.0,50,0.0,0.0,0.0,30.5,0.800570
4,Quarter1,sweing,Thursday,6.0,0.80,25.90,1170.0,1920.0,50,0.0,0.0,0.0,56.0,0.800382
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1192,Quarter2,finishing,Wednesday,10.0,0.75,2.90,NaN,960.0,0,0.0,0.0,0.0,8.0,0.628333
1193,Quarter2,finishing,Wednesday,8.0,0.70,3.90,NaN,960.0,0,0.0,0.0,0.0,8.0,0.625625
1194,Quarter2,finishing,Wednesday,7.0,0.65,3.90,NaN,960.0,0,0.0,0.0,0.0,8.0,0.625625
1195,NaN,finishing,Wednesday,9.0,0.75,2.90,NaN,1800.0,0,0.0,0.0,0.0,15.0,0.505889


The teams column appears to be categorical so we will inspect the number of distinct values.

In [ ]:
distinct_values = set(data['team'])

print(f"Distinct values in column 'B':")
for value in distinct_values:
    print(value)

Distinct values in column 'B':
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
nan
nan


## 2. Determine the type of each column

In [ ]:
data.dtypes

quarter                   object
department                object
day                       object
team                     float64
targeted_productivity    float64
smv                      float64
wip                      float64
over_time                float64
incentive                  int64
idle_time                float64
idle_men                 float64
no_of_style_change       float64
no_of_workers            float64
actual_productivity      float64
dtype: object

After inspecting the data, the clomuns can be clasified as follows;

**numeric:**
*   targeted_productivity
*   smv
*   wip
*   overtime
*   incentive
*   idle_time
*   idle_men
*   no_of_style_change
*   no_of_workers
*   actual_productivity

**categoric:**
*   quarter
*   department
*   day
*   team
(*team* though type float seem to be categorical since there are 12 consecutive numbers, and teams are generally named and not intrinsically ordered)



In [ ]:
numeric_columns = ["targeted_productivity", "smv", "wip", "overtime", "incentive", "idle_time", "idle_men", "no_of_style_change", "no_of_workers"]
categoric_columns = ["quarter", "department", "day", "team"]
target = ["actual_productivity"]

## 3. Split train/test

We'll split the data 80% for training and 20% for testing and give it a seed of 42 for consistency.

In [ ]:
from sklearn.model_selection import train_test_split
training_data, testing_data = train_test_split(data, test_size=0.2, random_state=42)
training_data.shape, testing_data.shape

((957, 14), (240, 14))

## 4. Create a pipeline

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

First we build the pipeline to transform the numerical columns. We will impute using the median and then apply a standard scalar.

In [ ]:
numerical_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('standardize', StandardScaler())
])

Next we will build the pipeline to transform the categorical columns. We will impute using the most frequent occurrence and then apply a one hot encoder.

In [ ]:
categorical_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('encode', OneHotEncoder(handle_unknown='ignore'))
])

Finally we apply these processing steps in a single pipeline to the respective columns which results in the final pipeline.

In [ ]:
processing_pipeline = ColumnTransformer(transformers=[
    ('numerical', numerical_pipeline, numeric_columns),
    ('categorical', categorical_pipeline, categoric_columns)
    ])

In [ ]:
processing_pipeline

ColumnTransformer(transformers=[('numerical',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardize',
                                                  StandardScaler())]),
                                 ['targeted_productivity', 'smv', 'wip',
                                  'overtime', 'incentive', 'idle_time',
                                  'idle_men', 'no_of_style_change',
                                  'no_of_workers']),
                                ('categorical',
                                 Pipeline(steps=[('impute',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encode',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['quarter', 'department', 'day', 'team'])])